# Object as source of rays and image loss

SourceObject
    an 'object' in the sense of optics that emits (really reflects) rays in all directions
    act as a source for the optical system
    properties: position, height

Ray:
    - object of origin?
    - absolute origin (object space coordinate of source)
    - during raytracing: origin / unit vector

TargetImage:
    - height
    - position

SourceDiffuse
    A diffuse light source that's not an object
    angular power function
    emits rays that are not associated with an object source

BeamLoss:
    Loss function for a beam of light spread,
    usefull to minimize beam spread

Object at infinity / image at infinity. ex: moon
    specific object
    specific image


OpticalObject
OpticalImage

OpticalObjectAtInfinity
OpticalImageAtInfinity

Moon / telescope example
    two rays coming from the same point on the object at infinity are //
    two rays coming from different points on the object are not //
    object at infinity doesn't have a size, but has angular size

In [ ]:
import torch
import torch.optim as optim
import torchlensmaker as tlm


class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        
        self.shape1 = tlm.CircularArc(height=60., r=tlm.Parameter(torch.tensor(120.)))
        self.shape2 = tlm.CircularArc(height=50., r=tlm.Parameter(torch.tensor(150.)))

        self.optics = tlm.OpticalSequence(
            tlm.ObjectAtInfinity(beam_diameter=40, angular_size=15),
            tlm.Gap(10.),
            tlm.Aperture(height=60, diameter=35),
            tlm.Gap(1.),
            tlm.SymmetricLens(self.shape1, n=(1.0, 1.5), outer_thickness=5.),
            tlm.Gap(1.),
            tlm.PlanoLens(self.shape2, n=(1.0, 1.5), outer_thickness=5.),
            tlm.Gap(60.0),
            tlm.Image(height=20),
        )

    def forward(self, inputs, sampling):
        return self.optics.forward(inputs, sampling)


optics = Optics()

tlm.render_plt(optics, sampling={"rays": 25, "object": 5}, color_dim="object")

output = optics(tlm.default_input, sampling={"rays": 10, "object": 3})
print("loss = ", output.loss.item())

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"rays": 10, "object": 5},
    num_iter = 200,
)

tlm.render_plt(optics, sampling={"rays": 25, "object": 5}, color_dim="object")


In [ ]:
## README example1
# currently not ready

import torchlensmaker as tlm

shape = tlm.CircularArc(height=60., r=70.)

optics = tlm.OpticalSequence(
    tlm.ObjectAtInfinity(beam_diameter=40, angular_size=15),
    tlm.Gap(10.),
    tlm.SymmetricLens(shape, (1.0, 1.5), outer_thickness=5.),
    tlm.Gap(60.0),
    tlm.Image(height=10),
)

tlm.render_plt(optics, sampling={"rays": 30, "object": 5}, color_dim="object")